In [140]:
import pandas as pd 
#read the file 
data = pd.read_csv("data/dataset-1.csv") 
data.head()

,state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,...,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [141]:
#Replace true and false by 1 and 0 respectively
data['churn'].replace(True, 1, inplace=True)
data['churn'].replace(False, 0, inplace=True)

In [142]:
data.head()

,state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,...,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0.0
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0.0
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0.0
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0.0
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0.0


In [143]:
#Discretization by binning
data['account length'] = pd.cut(data['account length'],10,right=True,labels=[0,1,2,3,4,5,6,7,8,9])
data['number vmail messages'] = pd.cut(data['number vmail messages'],10,right=True,labels=[0,1,2,3,4,5,6,7,8,9])
data['total day minutes'] = pd.cut(data['total day minutes'],10,right=True,labels=[0,1,2,3,4,5,6,7,8,9])
data['total day calls'] = pd.cut(data['total day calls'],10,right=True,labels=[0,1,2,3,4,5,6,7,8,9])
data['total day charge'] = pd.cut(data['total day charge'],10,right=True,labels=[0,1,2,3,4,5,6,7,8,9])
data['total eve minutes'] = pd.cut(data['total eve minutes'],10,right=True,labels=[0,1,2,3,4,5,6,7,8,9])
data['total eve calls'] = pd.cut(data['total eve calls'],10,right=True,labels=[0,1,2,3,4,5,6,7,8,9])
data['total eve charge'] = pd.cut(data['total eve charge'],10,right=True,labels=[0,1,2,3,4,5,6,7,8,9])
data['total night minutes'] = pd.cut(data['total night minutes'],10,right=True,labels=[0,1,2,3,4,5,6,7,8,9])
data['total night calls'] = pd.cut(data['total night calls'],10,right=True,labels=[0,1,2,3,4,5,6,7,8,9])
data['total night charge'] = pd.cut(data['total night charge'],10,right=True,labels=[0,1,2,3,4,5,6,7,8,9])
data['total intl minutes'] = pd.cut(data['total intl minutes'],10,right=True,labels=[0,1,2,3,4,5,6,7,8,9])
data['total intl calls'] = pd.cut(data['total intl calls'],10,right=True,labels=[0,1,2,3,4,5,6,7,8,9])
data['total intl charge'] = pd.cut(data['total intl charge'],10,right=True,labels=[0,1,2,3,4,5,6,7,8,9])

In [144]:
data['international plan'].replace('no', 0, inplace=True)
data['international plan'].replace('yes', 1, inplace=True)
data['voice mail plan'].replace('no', 0, inplace=True)
data['voice mail plan'].replace('yes', 1, inplace=True)

In [145]:
data.head()

,state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,...,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,KS,5,415,382-4657,0,1,4,7,6,7,...,5,5,5,4,5,4,1,4,1,0.0
1,OH,4,415,371-7191,0,1,5,4,7,4,...,6,5,6,4,6,6,1,6,1,0.0
2,NJ,5,415,358-1921,0,0,0,6,6,6,...,6,3,3,4,3,6,2,6,0,0.0
3,OH,3,408,375-9999,1,0,0,8,4,8,...,5,1,4,3,4,3,3,3,2,0.0
4,OK,3,415,330-6626,1,0,0,4,6,4,...,7,4,4,6,4,5,1,5,3,0.0


In [146]:
data.groupby('churn').count()

,state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls
churn,,,,,,,,,,,,,,,,,,,,
0.0,2850,2850,2850,2850,2850,2850,2850,2850,2850,2850,2850,2850,2850,2850,2850,2850,2850,2850,2850,2850
1.0,483,483,483,483,483,483,483,483,483,483,483,483,483,483,483,483,483,483,483,483


In [147]:
data = data.drop(columns=['phone number','area code','state'])

In [148]:
from scipy.spatial import distance
import numpy as np


matrix = distance.cdist(data.iloc[:,1:],data.iloc[:,1:], 'cityblock')
distanceMatrix = pd.DataFrame(matrix)

In [149]:
list(data)

['account length',
 'international plan',
 'voice mail plan',
 'number vmail messages',
 'total day minutes',
 'total day calls',
 'total day charge',
 'total eve minutes',
 'total eve calls',
 'total eve charge',
 'total night minutes',
 'total night calls',
 'total night charge',
 'total intl minutes',
 'total intl calls',
 'total intl charge',
 'customer service calls',
 'churn']

In [150]:
data.head()

,account length,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,5,0,1,4,7,6,7,5,5,5,5,4,5,4,1,4,1,0.0
1,4,0,1,5,4,7,4,5,6,5,6,4,6,6,1,6,1,0.0
2,5,0,0,0,6,6,6,3,6,3,3,4,3,6,2,6,0,0.0
3,3,1,0,0,8,4,8,1,5,1,4,3,4,3,3,3,2,0.0
4,3,1,0,0,4,6,4,4,7,4,4,6,4,5,1,5,3,0.0


In [156]:
data.loc[:,'distance'] = distanceMatrix.sum(axis=1)

AttributeError: 'Series' object has no attribute 'sort'